In [5]:
#-*- coding: utf-8 -*-
#!/usr/bin/env python


from nltk.text import Text
from nltk.corpus import state_union
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag.stanford import StanfordNERTagger as st
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn.manifold import MDS
from scipy.cluster.hierarchy import ward, dendrogram
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk
import re
import os  # for os.path.basename
#import urllib


stemmer = SnowballStemmer("english")# está entre porter y lancaster stemmer PONER EN LA TESIS EL PORQUE SE HA ELEGIDO ESTE.
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def COSINE(ref):#cit = query sentence ref = reference againts you match // Funcion tokenize_and_stem y tokenize_only va dentro de los parametros de TfidfVectorizer como "tokenizer = tokenize_and_stem" o "tokenizer = tokenize_only"
    pepe = {}
    winref = []
    documents = ref #before[0:]
    tfidf_vectorizer = TfidfVectorizer( stop_words='english',use_idf=True, tokenizer = tokenize_and_stem, min_df = 2)
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    CS = 1 - cosine_similarity(tfidf_matrix)
    return CS

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    lemma = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    stems = [stemmer.stem(t) for t in lemma]
    return stems

QUERY = """An alternative format, and one that has been widely used for Cas9-based screens, is pooled screening, in which pooled lentiviral libraries are transduced at a low multiplicity of infection (MOI) to ensure that most cells receive only one stably integrated RNA guide"""

file_txt = open('Statements','r+')
x = []
for i in file_txt:
    if i[0].isdigit() == True:
        x.append(i)
dist = COSINE(x)

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances
Sentence = "Sentence - "
Sentence = [Sentence+str(i) for i in range(1,46)]
fig, ax = plt.subplots(figsize=(20, 7)) # set size
ax = dendrogram(linkage_matrix, orientation="top", labels=Sentence);

plt.tick_params(\
    axis= 'x')        # changes apply to the x-axis
    #which='both',      # both major and minor ticks are affected
    #bottom='off',      # ticks along the bottom edge are off
    #top='off',         # ticks along the top edge are off
    #labelbottom='off')

plt.tight_layout() #show plot with tight layout
#uncomment below to save figure
plt.savefig('Statements(ward_clusters).png', dpi=300) 
#---------------------------------------------
###  MULTIDIMENSIONAL SCALING MDS
#MDS()

## convert two components as we're plotting points in a two-dimensional plane
## "precomputed" because we provide a distance matrix
## we will also specify `random_state` o the plot is reproducible.
#mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
#pos = mds.fit_transform(dist)  # shape (n_components, n_samples)
#xs, ys = pos[:, 0], pos[:, 1]


